# Multipool Tribrata News Sentiment Analysis

this function will get data from google_result table
and calculate the sentiment analysis. Result is store in google_result_sentiment

crontab command
* * * * * /home/haviz/multipool/ai-case-study/run-ai-sa-tbn.sh >> /home/haviz/multipool/ai-sa-tbn.log



In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import pymysql.cursors
import sqlalchemy as sa
import nltk
import string
import os

from nltk.corpus import stopwords 
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
import pandas as pd

# nltk.download('stopwords')


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")

In [2]:
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True

def execute_query(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

In [3]:
import pandas as pd
import re
from sqlalchemy import create_engine

def stemming(comment):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comment:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean
    
# function case folding
def casefolding(comment):
    comment = comment.lower()
    comment = comment.strip(" ")
    comment = re.sub(r'[?|$|.|!_:")(-+,]','',comment)
    return comment

def clean_up_tag(comment):
    x_ret = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",comment).split())
    return x_ret

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    
    # remove whitspace
    strOut = strOut.strip()
    
    # 
    strOut = re.sub('\s+',' ',strOut)
    return strOut

In [4]:
def process_by_df(source_ds):
    from tqdm import tqdm
    pd.options.mode.chained_assignment = None 

    # we calculate sentiment for 'title' and 'description' column

    # remove first
    source_ds['title'] = source_ds['title'].apply(clean_up_tag)
    source_ds['description'] = source_ds['content'].apply(clean_up_tag)


    # skip stemming
    source_ds['stemmed'] = source_ds['title']
    source_ds['stemmed'] = source_ds['stemmed'].apply(casefolding)
    source_ds['stemmed'] = source_ds['stemmed'].apply(text_preproc)

    sw = stopwords.words('indonesian')
    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)
    sw = stopwords.words('indonesian')

    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay

    # Create a new list and insert each element from the original list
    from tqdm import tqdm

    list_text = source_ds['stemmed'].tolist()
    new_list = []
    for i in tqdm(range( len(list_text) )):
        res = pipe(list_text[i])
        #print(res)
        new_list.append(res)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay
    # create a list of our conditions
    for i in range(0, len(new_list)):
        # print(new_list[i][0]['label'])
        if new_list[i][0]['label'] == 'Positive':
            new_list[i][0]['Value'] = 1
        elif new_list[i][0]['label'] == 'Negative':
            new_list[i][0]['Value'] = -1
        elif new_list[i][0]['label'] == 'Neutral':
            new_list[i][0]['Value'] = 0

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']
        source_ds.at[index,'Score'] = new_list[index][0]['score']
        #source_ds.at[index,'Score'] = new_list[index][0]['Value']
        
    return source_ds


In [5]:
def process_by_df_col(source_ds, col_name):
    from tqdm import tqdm
    pd.options.mode.chained_assignment = None 
    
    # trim to 2500 character
    source_ds[col_name] = source_ds[col_name].str.slice(0,2000)

    # skip stemming
    source_ds['stemmed'] = source_ds[col_name].apply(clean_up_tag)
    source_ds['stemmed'] = source_ds['stemmed'].apply(casefolding)
    source_ds['stemmed'] = source_ds['stemmed'].apply(text_preproc)

    sw = stopwords.words('indonesian')
    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)
    sw = stopwords.words('indonesian')

    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay

    # Create a new list and insert each element from the original list
    from tqdm import tqdm

    list_text = source_ds['stemmed'].tolist()
    new_list = []
    for i in tqdm(range( len(list_text) )):
        # print(len(list_text[i]))
        try:
            if len(list_text[i]) > 0:
                res = pipe( list_text[i] )
                new_list.append(res)
            else:
                res = pipe( 'kalimat netral' )
                new_list.append(res)
                zip
        except Exception as e:
            # Handle the exception (error) here
            print(f"Error processing value at index {i}: {list_text[i]}")
            print(f"Error message: {str(e)}")
            res = pipe( 'kalimat netral' )
            new_list.append(res)      
            
            continue

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay
    # create a list of our conditions
    for i in range(0, len(new_list)):
        # print(new_list[i][0]['label'])
        if new_list[i][0]['label'] == 'Positive':
            new_list[i][0]['Value'] = 1
        elif new_list[i][0]['label'] == 'Negative':
            new_list[i][0]['Value'] = -1
        elif new_list[i][0]['label'] == 'Neutral':
            new_list[i][0]['Value'] = 0

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']
        source_ds.at[index,'Score'] = new_list[index][0]['score']
        #source_ds.at[index,'Score'] = new_list[index][0]['Value']
        
    return source_ds

In [6]:
# query 400 rows of data query data from table
select_query = "select * \
from news_tbn_medol gr \
where id not in (select id_tbn_news from news_tbn_sentiment) \
order by id desc limit 500"

source_ds = execute_query(select_query)

if len(source_ds) == 0:
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()
    
# source_ds.head()

In [7]:
# Processing title sentiment analysis
content_ds = source_ds
res_ds = process_by_df(source_ds)

  0%|          | 0/43 [00:00<?, ?it/s]

  2%|▏         | 1/43 [00:00<00:21,  1.94it/s]

  7%|▋         | 3/43 [00:00<00:07,  5.31it/s]

 12%|█▏        | 5/43 [00:00<00:04,  7.85it/s]

 16%|█▋        | 7/43 [00:00<00:03,  9.69it/s]

...

 86%|████████▌ | 37/43 [00:03<00:00, 14.01it/s]

 91%|█████████ | 39/43 [00:03<00:00, 14.64it/s]

 95%|█████████▌| 41/43 [00:03<00:00, 14.46it/s]

100%|██████████| 43/43 [00:03<00:00, 15.11it/s]

100%|██████████| 43/43 [00:03<00:00, 12.41it/s]

In [8]:
res_ds.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,tbn_medol_id,is_polri,reporter,editor,converted_date,description,stemmed,tokenized_tweet,Prediction,Score
0,6566,Polda Sulbar Buka Hotline Pengaduan Masalah Re...,Tribratanews.polri.go.id - Mamuju. Biro Sumber...,5 June 2024 - 19:30 WIB,,https://tribratanews.polri.go.id/,https://tribratanews.polri.go.id/blog/nasional...,2024-06-05 12:59:46.103697,35,True,,mz/pr/nm,2024-06-05 19:30:00,Tribratanews polri go id Mamuju Biro Sumber Da...,polda sulbar buka hotline pengaduan masalah re...,"{pengaduan, buka, rekrutmen, polri, hotline, s...",Neutral,0.999653
1,6565,Presiden Jokowi Meyakini Kualitas Udara IKN Le...,Tribratanews.polri.go.id - Jakarta. Presiden J...,5 June 2024 - 19:00 WIB,,https://tribratanews.polri.go.id/,https://tribratanews.polri.go.id/blog/nasional...,2024-06-05 12:59:46.095435,35,True,,ndt/hn/nm,2024-06-05 19:00:00,Tribratanews polri go id Jakarta Presiden Joko...,presiden jokowi meyakini kualitas udara ikn le...,"{melbourne, presiden, paris, kualitas, jokowi,...",Positive,0.991976
2,6564,Gempa Magnitudo 4 7 Guncang Wilayah Kota Bima NTB,Tribratanews.polri.go.id - Info gempa hari ini...,5 June 2024 - 18:50 WIB,,https://tribratanews.polri.go.id/,https://tribratanews.polri.go.id/blog/nasional...,2024-06-05 12:59:46.086984,35,True,,ek/hn/nm,2024-06-05 18:50:00,Tribratanews polri go id Info gempa hari ini B...,gempa magnitudo guncang wilayah kota bima ntb,"{gempa, guncang, magnitudo, bima, ntb, wilayah...",Neutral,0.999732
3,6563,Indonesia Open 2024 Leo Danie Menang dari Faja...,Tribratanews.polri.go.id - Jakarta. Leo Rolly ...,5 June 2024 - 18:48 WIB,,https://tribratanews.polri.go.id/,https://tribratanews.polri.go.id/blog/olahraga...,2024-06-05 12:59:46.078943,35,True,,rz/hn/nm,2024-06-05 18:48:00,Tribratanews polri go id Jakarta Leo Rolly Car...,indonesia open leo danie menang dari fajar rian,"{menang, fajar, open, indonesia, rian, danie, ...",Neutral,0.999582
4,6562,Presiden Jokowi Optimistis Bisa Berkantor di I...,Tribratanews.polri.go.id - Jakarta. Presiden J...,5 June 2024 - 18:30 WIB,,https://tribratanews.polri.go.id/,https://tribratanews.polri.go.id/blog/nasional...,2024-06-05 12:59:46.070067,35,True,,ndt/pr/nm,2024-06-05 18:30:00,Tribratanews polri go id Jakarta Presiden Joko...,presiden jokowi optimistis bisa berkantor di i...,"{juli, optimistis, presiden, berkantor, jokowi...",Neutral,0.992382


In [9]:
# Processing content sentiment analysis
res_content_ds = process_by_df_col(content_ds,'content')

  0%|          | 0/43 [00:00<?, ?it/s]

  2%|▏         | 1/43 [00:00<00:13,  3.11it/s]

  5%|▍         | 2/43 [00:00<00:10,  3.74it/s]

  7%|▋         | 3/43 [00:00<00:08,  4.59it/s]

  9%|▉         | 4/43 [00:00<00:07,  5.33it/s]

...

 93%|█████████▎| 40/43 [00:08<00:00,  5.04it/s]

 95%|█████████▌| 41/43 [00:09<00:00,  4.86it/s]

 98%|█████████▊| 42/43 [00:09<00:00,  5.21it/s]

100%|██████████| 43/43 [00:09<00:00,  4.97it/s]

100%|██████████| 43/43 [00:09<00:00,  4.52it/s]

In [10]:
res_content_ds.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,tbn_medol_id,is_polri,reporter,editor,converted_date,description,stemmed,tokenized_tweet,Prediction,Score
0,6566,Polda Sulbar Buka Hotline Pengaduan Masalah Re...,Tribratanews.polri.go.id - Mamuju. Biro Sumber...,5 June 2024 - 19:30 WIB,,https://tribratanews.polri.go.id/,https://tribratanews.polri.go.id/blog/nasional...,2024-06-05 12:59:46.103697,35,True,,mz/pr/nm,2024-06-05 19:30:00,Tribratanews polri go id Mamuju Biro Sumber Da...,tribratanews polri go id mamuju biro sumber da...,"{pungky, pembangunan, langsung, data, luasnya,...",Neutral,0.999186
1,6565,Presiden Jokowi Meyakini Kualitas Udara IKN Le...,Tribratanews.polri.go.id - Jakarta. Presiden J...,5 June 2024 - 19:00 WIB,,https://tribratanews.polri.go.id/,https://tribratanews.polri.go.id/blog/nasional...,2024-06-05 12:59:46.095435,35,True,,ndt/hn/nm,2024-06-05 19:00:00,Tribratanews polri go id Jakarta Presiden Joko...,tribratanews polri go id jakarta presiden joko...,"{membandingkan, ditopang, joko, melbourne, pre...",Neutral,0.997196
2,6564,Gempa Magnitudo 4 7 Guncang Wilayah Kota Bima NTB,Tribratanews.polri.go.id - Info gempa hari ini...,5 June 2024 - 18:50 WIB,,https://tribratanews.polri.go.id/,https://tribratanews.polri.go.id/blog/nasional...,2024-06-05 12:59:46.086984,35,True,,ek/hn/nm,2024-06-05 18:50:00,Tribratanews polri go id Info gempa hari ini B...,tribratanews polri go id info gempa hari ini b...,"{tsunami, selatan, jarak, aftershock, kota, kl...",Neutral,0.999654
3,6563,Indonesia Open 2024 Leo Danie Menang dari Faja...,Tribratanews.polri.go.id - Jakarta. Leo Rolly ...,5 June 2024 - 18:48 WIB,,https://tribratanews.polri.go.id/,https://tribratanews.polri.go.id/blog/olahraga...,2024-06-05 12:59:46.078943,35,True,,rz/hn/nm,2024-06-05 18:48:00,Tribratanews polri go id Jakarta Leo Rolly Car...,tribratanews polri go id jakarta leo rolly car...,"{rolly, langsung, laga, carnando, memimpin, al...",Neutral,0.998022
4,6562,Presiden Jokowi Optimistis Bisa Berkantor di I...,Tribratanews.polri.go.id - Jakarta. Presiden J...,5 June 2024 - 18:30 WIB,,https://tribratanews.polri.go.id/,https://tribratanews.polri.go.id/blog/nasional...,2024-06-05 12:59:46.070067,35,True,,ndt/pr/nm,2024-06-05 18:30:00,Tribratanews polri go id Jakarta Presiden Joko...,tribratanews polri go id jakarta presiden joko...,"{pembangunan, pasokan, kota, menaikkan, kantor...",Neutral,0.993217


In [11]:
# record result
sentiment_class = 0

for index, row in res_ds.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    # sql = "INSERT INTO cekmedsos_database.ret_sentiment_result (job_id, tweet_id, sentiment_class) VALUES(%s, %s , %s);" % (str(const_job_id), row['id_str'], str(sentiment_class))
    sql = "INSERT INTO public.news_tbn_sentiment (id_tbn_news, sentiment_title, sentiment_title_value) VALUES( %s, %s, %s);" % (row['id'], sentiment_class, row['Score'],) 
    #print(sql)
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.


In [12]:
# record result
sentiment_class = 0

for index, row in res_ds.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    sql = "UPDATE public.news_tbn_sentiment set sentiment_content = %s, sentiment_content_value = %s where id_tbn_news = %s;" % (sentiment_class, row['Score'], row['id'])
    
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


...

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
